# Увеличение производительности библиотеки Pandas: eval() и query()

Основные возможности стека PyData основываются на умении библиотек NumPy и Pandas передавать простые операции на выполнение программам на языке C посредством интуитивно понятного синтаксиса: примерами могут послужить векторизованные/транслируемые операции в библиотеке NumPy, а также операции группировки в библиотеке Pandas. Хотя эти абстракции весьма производительны и эффективно работают для многих распространенных сценариев использования, они зачастую требуют создания временных вспомогательных объектов, что приводит к чрезмерным накладным расходам как процессорного времени, так и оперативной памяти.

По состоянию на версию 0.13 (выпущенную в январе 2014 года) библиотека Pandas включает некоторые экспериментальные инструменты, позволяющие обращаться к работающим со скоростью написанных на языке C операциям без выделения существенных объемов памяти на промежуточные массивы. Эти утилиты — функции eval() и query(), основанные на пакете Numexpr (https://github.com/pydata/numexpr). Мы рассмотрим их использование и приведем некоторые эмпирические правила, позволяющие решить, имеет ли смысл их применять.

## Основания для использования функций query() и eval(): составные выражения

In [19]:
import numpy as np

In [21]:
x = 1E6
y = 1E6
mask = (x > 0.5) & (y < 0.5)
mask

False

Поскольку библиотека NumPy вычисляет каждое подвыражение, оно эквивалентно следующему:

In [23]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

Другими словами, для каждого промежуточного шага явным образом выделяется оперативная память. Если массивы x и y очень велики, это может привести к значительным накладным расходам оперативной памяти и процессорного времени.

Библиотека Numexpr позволяет вычислять подобные составные выражения поэлементно, не требуя выделения памяти под промежуточные массивы целиком. В документации библиотеки Numexpr (https://github.com/pydata/numexpr) приведено больше подробностей, но пока достаточно будет сказать, что функции этой библиотеки принимают на входе строку, содержащую выражение в стиле библиотеки NumPy, которое требуется вычислить:

In [26]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

Преимущество заключается в том, что библиотека Numexpr вычисляет выражение, не используя полноразмерных временных массивов, а потому оказывается намного более эффективной, чем NumPy, особенно в случае больших массивов. Инструменты query() и eval(), которые мы будем обсуждать, идеологически схожи и используют пакет Numexpr.

## Использование функции pandas.eval() для эффективных операций

Функция eval() библиотеки Pandas применяет строковые выражения для эффективных вычислительных операций с объектами DataFrame. Например, рассмотрим следующие объекты DataFrame:

In [27]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

Для вычисления суммы всех четырех объектов DataFrame при стандартном подходе библиотеки Pandas можно написать сумму:

In [29]:
%timeit df1 + df2 + df3 + df4

129 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Можно вычислить тот же результат с помощью функции pd.eval(), задав выражение в виде строки:

In [30]:
%timeit pd.eval('df1 + df2 + df3 + df4')

59.3 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Версия этого выражения с функцией eval() работает на 50 % быстрее (и использует намного меньше памяти), возвращая тот же самый результат:

In [31]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

### Поддерживаемые функцией pd.eval() операции

На момент выпуска версии 0.16 библиотеки Pandas функция pd.eval() поддерживает широкий спектр операций. Для их демонстрации мы будем использовать следующие целочисленные объекты DataFrame:

In [34]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) for i in range(5))

__Арифметические операторы.__ Функция pd.eval() поддерживает все арифметические операторы. Например:

In [36]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

__Операторы сравнения.__ Функция pd.eval() поддерживает все операторы сравнения, включая выражения, организованные цепочкой:

In [37]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

__Побитовые операторы.__ Функция pd.eval() поддерживает побитовые операторы & и |:

In [38]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

Кроме того, она допускает использование литералов and и or в булевых выражениях:

In [39]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

__Атрибуты объектов и индексы.__ Функция pd.eval() поддерживает доступ к атрибутам объектов с помощью синтаксиса obj.attr и к индексам посредством синтаксиса obj[index]:

In [40]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

__Другие операции.__ Другие операции, например вызовы функций, условные выражения, циклы и другие, более сложные конструкции, пока в функции pd.eval() не реализованы. При необходимости выполнения подобных сложных видов выражений можно воспользоваться самой библиотекой Numexpr.

## Использование метода DataFrame.eval() для выполнения операций по столбцам

У объектов DataFrame существует метод eval(), работающий схожим образом с высокоуровневой функцией pd.eval() из библиотеки Pandas. Преимущество метода eval() заключается в возможности ссылаться на столбцы _по имени_. Возьмем для примера следующий маркированный массив:

In [41]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789
...,...,...,...
995,0.082646,0.036840,0.439733
996,0.008826,0.896578,0.723374
997,0.907270,0.916424,0.978655
998,0.758995,0.535431,0.347766


Воспользовавшись функцией pd.eval() так, как показано выше, можно вычислять выражения с этими тремя столбцами следующим образом:

In [42]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

Обратите внимание, что мы обращаемся с названиями столбцов в вычисляемом выражении как с переменными и получаем желаемый результат.

## Присваивание в методе DataFrame.eval()

Метод DataFrame.eval() позволяет выполнять присваивание значения любому из столбцов. Воспользуемся предыдущим объектом DataFrame со столбцами 'A', 'B' и 'C':

In [43]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Метод DataFrame.eval() можно использовать, например, для создания нового столбца 'D' и присваивания ему значения, вычисленного на основе других столбцов:

In [44]:
df.eval('D= (A + B) / C', inplace=True)

In [46]:
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


Аналогично можно модифицировать значения любого уже существующего столбца:

In [47]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### Локальные переменные в методе DataFrame.eval()

Метод DataFrame.eval() поддерживает дополнительный синтаксис для работы с локальными переменными языка Python. Взгляните на следующий фрагмент кода:

In [49]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

Символ @ отмечает имя переменной, а не имя столбца, позволяя тем самым эффективно вычислять значение выражений с использованием двух пространств имен: пространства имен столбцов и пространства имен объектов Python. Обратите внимание, что этот символ @ поддерживается лишь методом DataFrame.eval(), но не функцией pandas.eval(), поскольку у функции pandas.eval() есть доступ только к одному пространству имен (языка Python).

## Метод DataFrame.query()

У объектов DataFrame есть еще один метод, основанный на вычислении строк, именуемый query(). Рассмотрим следующее:

In [50]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

Как и в случае с примером, который мы использовали при обсуждении метода DataFrame.eval(), перед нами выражение, содержащее столбцы объекта DataFrame. Однако его нельзя выразить с помощью синтаксиса метода DataFrame.eval()! Вместо него для подобных операций фильтрации можно воспользоваться методом query():

In [51]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

Он не только обеспечивает, по сравнению с выражениями маскирования, более эффективные вычисления, но и намного понятнее и легче читается. Обратите внимание, что метод query() также позволяет использовать флаг @ для обозначения локальных переменных:

In [55]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Производительность: когда следует использовать эти функции

В процессе принятия решения применять ли эти функции обратите внимание на два момента: процессорное время и объем используемой памяти. Предсказать объем используемой памяти намного проще. Как уже упоминалось, все составные выражения с применением массивов NumPy или объектов DataFrame библиотеки Pandas приводят к неявному созданию временных массивов. Например, вот это:

In [56]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

приблизительно соответствует следующему

In [57]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

Если размер временных объектов DataFrame существенен по сравнению с доступной оперативной памятью вашей системы (обычно несколько гигабайтов), то будет разумно воспользоваться выражениями eval() или query(). Выяснить приблизительный размер массива в байтах можно с помощью следующего оператора:

In [58]:
df.values.nbytes

32000

eval() будет работать быстрее, если вы не используете всю доступную в системе оперативную память. Основную роль играет отношение размера временных объектов DataFrame по сравнению с размером L1 или L2 кэша процессора в системе (в 2016 году он составляет несколько мегабайтов). eval() позволяет избежать потенциально медленного перемещения значений между различными кэшами памяти в том случае, когда это отношение намного больше 1. Я обнаружил, что на практике различие в скорости вычислений между традиционными методами и методом eval/query обычно довольно незначительно. Напротив, традиционный метод работает быстрее для маленьких массивов! Преимущество метода eval/query заключается в экономии оперативной памяти и иногда — в более понятном синтаксисе.

Мы рассмотрели большинство нюансов работы с методами eval() и query(), дополнительную информацию можно найти в документации по библиотеке Pandas. В частности, можно задавать для работы этих запросов различные синтаксические анализаторы и механизмы. Подробности — в разделе Enhancing Performance («Повышение производительности», http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html).